## Notebook 2: Watching Crops Grow from Space

**Can we watch crops grow from space?**

In the previous notebook, we computed a single NDVI snapshot. But vegetation changes throughout the year. In this notebook, we'll:

- Create monthly NDVI composites across the growing season
- Compare early vs. peak season side-by-side
- Generate an animated GIF showing vegetation change over time

## Setup

Same initialization as Notebook 01.

In [1]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
from google.cloud import storage
from IPython.display import Image, display

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

Ready!


## Define area of interest

We'll use Missouri again from the TIGER dataset.

In [3]:
states = ee.FeatureCollection("TIGER/2018/States")
mo = states.filter(ee.Filter.eq("NAME", "Missouri")).geometry()

## Cloud mask and NDVI functions

Same functions from Notebook 01 - masking clouds and computing NDVI.

In [4]:
def mask_s2_clouds(img):
    """Mask clouds using Sentinel-2 Scene Classification Layer."""
    scl = img.select("SCL")
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return img.updateMask(mask)

def add_ndvi(img):
    """Add NDVI band to image."""
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

# NDVI color palette
NDVI_PALETTE = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
NDVI_VIS = {"min": 0.0, "max": 0.8, "palette": NDVI_PALETTE}

## Create monthly NDVI composites

We'll generate one NDVI composite per month from April through October 2024 - the agricultural growing season in Missouri.

In [5]:
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Months to process (Apr-Oct 2024)
months = [
    ("2024-04-01", "2024-04-30", "April"),
    ("2024-05-01", "2024-05-31", "May"),
    ("2024-06-01", "2024-06-30", "June"),
    ("2024-07-01", "2024-07-31", "July"),
    ("2024-08-01", "2024-08-31", "August"),
    ("2024-09-01", "2024-09-30", "September"),
    ("2024-10-01", "2024-10-31", "October"),
]

def get_monthly_ndvi(start, end, label):
    """Create median NDVI composite for a date range."""
    composite = (
        s2.filterBounds(mo)
          .filterDate(start, end)
          .map(mask_s2_clouds)
          .map(add_ndvi)
          .select("NDVI")
          .median()
          .clip(mo)
    )
    return composite.set("month", label)

# Build list of monthly composites
monthly_ndvi = [get_monthly_ndvi(start, end, label) for start, end, label in months]

print(f"Created {len(monthly_ndvi)} monthly composites")

Created 7 monthly composites


## Compare April vs. July

Early spring vs. peak growing season - you should see a dramatic difference in vegetation.

In [6]:
# April (index 0) vs July (index 3)
april_ndvi = monthly_ndvi[0]
july_ndvi = monthly_ndvi[3]

# Create side-by-side maps
left_layer = geemap.ee_tile_layer(april_ndvi, NDVI_VIS, "April 2024")
right_layer = geemap.ee_tile_layer(july_ndvi, NDVI_VIS, "July 2024")

m = geemap.Map(center=[38.5, -92.5], zoom=7)
m.split_map(left_layer, right_layer)
m

## Generate animated GIF

Now let's create an animation showing all seven months. We'll use Earth Engine's `getVideoThumbURL()` to generate the GIF server-side.

In [7]:
# Convert list to ImageCollection for animation
ndvi_collection = ee.ImageCollection.fromImages(monthly_ndvi)

# Get the bounding box for Missouri
mo_bounds = mo.bounds().getInfo()['coordinates'][0]
region = ee.Geometry.Rectangle([mo_bounds[0], mo_bounds[2]])

# Animation parameters
animation_params = {
    'region': region,
    'dimensions': 600,
    'crs': 'EPSG:4326',
    'framesPerSecond': 1,
    'min': 0.0,
    'max': 0.8,
    'palette': NDVI_PALETTE
}

# Generate the animation URL
gif_url = ndvi_collection.getVideoThumbURL(animation_params)
print("Animation URL generated!")
print(gif_url[:80] + "...")

Animation URL generated!
https://earthengine.googleapis.com/v1/projects/eeps-geospatial/videoThumbnails/1...


## Display the animation

Watch Missouri green up from April through July, then begin to brown as crops mature and are harvested.

In [8]:
display(Image(url=gif_url))

## Try it yourself

Ideas to extend this notebook:

1. **Different state** - Change the filter to another agricultural state (Iowa, Kansas, Nebraska)
2. **Different year** - Compare 2024 to a drought year
3. **Zoom in** - Pick a specific county or draw a custom region
4. **Change the speed** - Adjust `framesPerSecond` in the animation parameters
5. **Export the GIF** - Save the animation to Google Cloud Storage for use elsewhere